In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df_full = pd.read_csv('../data/LoanStats2014.csv', header=1, skip_footer=2)

C:\Users\Daniel\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: The 'skip_footer' argument has been deprecated and will be removed in a future version. Please use the 'skipfooter' argument instead.
  if __name__ == '__main__':
C:\Users\Daniel\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


In [3]:
df = df_full.copy()

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235629 entries, 0 to 235628
Columns: 111 entries, id to total_il_high_credit_limit
dtypes: float64(44), int64(43), object(24)
memory usage: 199.5+ MB


### Define targets

</br> Initially, try a binary target... <br />1 if total payment received is >= 1.05% of amount funded<br />0 if loan status is 'charged off', 'default', '31-120 days late', or is fully paid, but not >= 1.05% of funded

In [5]:
def final_status(status, funded_amnt, total_pymnt):
    #define list to drop
    drop_status = ['In Grace Period', 'Late (16-30 days)']
    neg_status = ['Late (31-120 days)', 'Charged Off', 'Default']
    
    if status in drop_status:
        return 99
    elif status in neg_status:
        return 0
    elif status == 'Fully Paid':
        if total_pymnt>funded_amnt*1.05:
            return 1
        else:
            return 0
    elif status == 'Current':
        if total_pymnt>funded_amnt*1.05:
            return 1
        else:
            return 99     

In [6]:
df['target'] = df.apply(lambda x: final_status(x['loan_status'], x['funded_amnt_inv'], x['total_pymnt_inv']), axis=1)

In [7]:
df['target'].value_counts()

99    100850
1      93301
0      41478
Name: target, dtype: int64

In [8]:
df.drop(df[df['target']==99].index, inplace=True)

In [9]:
df.reset_index(inplace=True, drop=True)

In [11]:
df.set_index(['id'], inplace=True)

In [13]:
df.head()

,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,...,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,target
id,,,,,,,,,,,,,,,,,,,,,
36805548,39558264,10400,10400,10400,36 months,6.99%,321.08,A,A3,Truck Driver Delivery Personel,...,4,83.3,14.3,0,0,179407,15030,13000,11325,0
38098114,40860827,15000,15000,15000,60 months,12.39%,336.64,C,C1,MANAGEMENT,...,4,100.0,0.0,0,0,196500,149140,10000,12000,1
37822187,40585251,9600,9600,9600,36 months,13.66%,326.53,C,C3,Admin Specialist,...,3,100.0,60.0,0,0,52490,38566,21100,24890,0
37662224,40425321,7650,7650,7650,36 months,13.66%,260.20,C,C3,Technical Specialist,...,2,100.0,100.0,0,0,82331,64426,4900,64031,0
37842129,40605224,21425,21425,21425,60 months,15.59%,516.36,D,D1,Programming Analysis Supervisor,...,2,91.4,100.0,0,0,57073,42315,15000,35573,1


## Clean Data

In [16]:
n_cols = df.shape[1]

In [17]:
drop_cols = [
        'url'
        , 'application_type'
        , 'policy_code'
    ]

In [18]:
df.drop(drop_cols, axis=1, inplace=True)

In [19]:
df.dropna(how='all', axis=1, inplace=True)

### Choose only columns we will have information for when loan originates

In [20]:
for i in df.columns:
    print ", '{}'".format(i)

, 'member_id'
, 'loan_amnt'
, 'funded_amnt'
, 'funded_amnt_inv'
, 'term'
, 'int_rate'
, 'installment'
, 'grade'
, 'sub_grade'
, 'emp_title'
, 'emp_length'
, 'home_ownership'
, 'annual_inc'
, 'verification_status'
, 'issue_d'
, 'loan_status'
, 'pymnt_plan'
, 'desc'
, 'purpose'
, 'title'
, 'zip_code'
, 'addr_state'
, 'dti'
, 'delinq_2yrs'
, 'earliest_cr_line'
, 'inq_last_6mths'
, 'mths_since_last_delinq'
, 'mths_since_last_record'
, 'open_acc'
, 'pub_rec'
, 'revol_bal'
, 'revol_util'
, 'total_acc'
, 'initial_list_status'
, 'out_prncp'
, 'out_prncp_inv'
, 'total_pymnt'
, 'total_pymnt_inv'
, 'total_rec_prncp'
, 'total_rec_int'
, 'total_rec_late_fee'
, 'recoveries'
, 'collection_recovery_fee'
, 'last_pymnt_d'
, 'last_pymnt_amnt'
, 'next_pymnt_d'
, 'last_credit_pull_d'
, 'collections_12_mths_ex_med'
, 'mths_since_last_major_derog'
, 'acc_now_delinq'
, 'tot_coll_amt'
, 'tot_cur_bal'
, 'total_rev_hi_lim'
, 'acc_open_past_24mths'
, 'avg_cur_bal'
, 'bc_open_to_buy'
, 'bc_util'
, 'chargeoff_withi

In [21]:
keep_cols = [
            #, 'id'
            #, 'member_id'
            'loan_amnt'
            #, 'funded_amnt'
            #, 'funded_amnt_inv'
            , 'term'
            , 'int_rate'
            , 'installment'
            , 'grade'
            , 'sub_grade'
            #, 'emp_title'
            , 'emp_length'
            , 'home_ownership'
            , 'annual_inc'
            , 'verification_status'
            #, 'issue_d'
            #, 'loan_status'
            #, 'pymnt_plan'
            , 'desc'
            , 'purpose'
            #, 'title'
            #, 'zip_code'
            #, 'addr_state'
            , 'dti'
            , 'delinq_2yrs'
            , 'earliest_cr_line'
            , 'inq_last_6mths'
            , 'mths_since_last_delinq'
            , 'mths_since_last_record'
            , 'open_acc'
            , 'pub_rec'
            , 'revol_bal'
            , 'revol_util'
            , 'total_acc'
            #, 'initial_list_status'
            #, 'out_prncp'
            #, 'out_prncp_inv'
            #, 'total_pymnt'
            #, 'total_pymnt_inv'
            #, 'total_rec_prncp'
            #, 'total_rec_int'
            #, 'total_rec_late_fee'
            #, 'recoveries'
            #, 'collection_recovery_fee'
            #, 'last_pymnt_d'
            #, 'last_pymnt_amnt'
            #, 'next_pymnt_d'
            #, 'last_credit_pull_d'
            , 'collections_12_mths_ex_med'
            , 'mths_since_last_major_derog'
            , 'acc_now_delinq'
            #, 'tot_coll_amt'
            #, 'tot_cur_bal'
            , 'total_rev_hi_lim'
            , 'acc_open_past_24mths'
            , 'avg_cur_bal'
            , 'bc_open_to_buy'
            , 'bc_util'
            , 'chargeoff_within_12_mths'
            , 'delinq_amnt'
            , 'mo_sin_old_il_acct'
            , 'mo_sin_old_rev_tl_op'
            , 'mo_sin_rcnt_rev_tl_op'
            , 'mo_sin_rcnt_tl'
            , 'mort_acc'
            , 'mths_since_recent_bc'
            , 'mths_since_recent_bc_dlq'
            , 'mths_since_recent_inq'
            , 'mths_since_recent_revol_delinq'
            , 'num_accts_ever_120_pd'
            , 'num_actv_bc_tl'
            , 'num_actv_rev_tl'
            , 'num_bc_sats'
            , 'num_bc_tl'
            , 'num_il_tl'
            , 'num_op_rev_tl'
            , 'num_rev_accts'
            , 'num_rev_tl_bal_gt_0'
            , 'num_sats'
            , 'num_tl_120dpd_2m'
            , 'num_tl_30dpd'
            , 'num_tl_90g_dpd_24m'
            , 'num_tl_op_past_12m'
            , 'pct_tl_nvr_dlq'
            , 'percent_bc_gt_75'
            , 'pub_rec_bankruptcies'
            , 'tax_liens'
            , 'tot_hi_cred_lim'
            , 'total_bal_ex_mort'
            , 'total_bc_limit'
            , 'total_il_high_credit_limit'
            ]

In [22]:
df = df[keep_cols]

### Turn into correct data type

In [23]:
df.select_dtypes(exclude=[np.number]).T

id,36805548,38098114,37822187,37662224,37842129,37682226,36804663,37741884,37800722,37642222,...,9199716,9666300,9715141,9695736,9007579,9745590,9745484,9684700,9604874,9199665
term,36 months,60 months,36 months,36 months,60 months,36 months,36 months,36 months,36 months,36 months,...,36 months,36 months,36 months,60 months,36 months,36 months,36 months,60 months,36 months,36 months
int_rate,6.99%,12.39%,13.66%,13.66%,15.59%,13.66%,14.31%,11.99%,17.86%,11.44%,...,10.99%,11.99%,15.61%,18.25%,7.90%,18.25%,13.53%,19.97%,7.90%,19.20%
grade,A,C,C,C,D,C,C,B,D,B,...,B,B,C,D,A,D,B,D,A,D
sub_grade,A3,C1,C3,C3,D1,C3,C4,B5,D5,B4,...,B2,B3,C4,D3,A4,D3,B5,D5,A4,D3
emp_length,8 years,10+ years,10+ years,< 1 year,6 years,10+ years,10+ years,< 1 year,10+ years,2 years,...,n/a,8 years,7 years,5 years,3 years,10+ years,4 years,10+ years,3 years,10+ years
home_ownership,MORTGAGE,RENT,RENT,RENT,RENT,MORTGAGE,RENT,MORTGAGE,RENT,RENT,...,MORTGAGE,RENT,RENT,MORTGAGE,RENT,RENT,RENT,MORTGAGE,OWN,MORTGAGE
verification_status,Not Verified,Source Verified,Source Verified,Source Verified,Source Verified,Verified,Source Verified,Source Verified,Source Verified,Not Verified,...,Verified,Source Verified,Verified,Verified,Verified,Source Verified,Source Verified,Verified,Verified,Verified
desc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Borrower added on 12/10/13 > all my loans in...,Borrower added on 12/10/13 > pay off higher ...,NaN,NaN,Borrower added on 12/09/13 > consolidate all...,NaN,NaN,NaN,NaN,Borrower added on 12/04/13 > I will like a l...
purpose,credit_card,debt_consolidation,debt_consolidation,debt_consolidation,credit_card,debt_consolidation,credit_card,home_improvement,house,debt_consolidation,...,credit_card,credit_card,credit_card,medical,debt_consolidation,vacation,credit_card,debt_consolidation,credit_card,other
earliest_cr_line,Sep-1989,Aug-1994,Nov-1992,Aug-2002,Aug-2003,Jan-2001,Oct-1998,Oct-2001,Jan-1999,May-2009,...,May-1990,Aug-1983,Nov-1993,Nov-1994,Jul-1998,Jun-2002,Aug-1999,Jun-1997,Feb-2003,Feb-2000


In [24]:
for i in df.select_dtypes(exclude=[np.number]).columns:
    print ", '{0}'".format(i)

, 'term'
, 'int_rate'
, 'grade'
, 'sub_grade'
, 'emp_length'
, 'home_ownership'
, 'verification_status'
, 'desc'
, 'purpose'
, 'earliest_cr_line'
, 'revol_util'


#### Object to numeric

In [25]:
import re

In [26]:
obj_num_cols = [
        'term'
        , 'int_rate'
        , 'emp_length'
        , 'revol_util'
        ]

In [27]:
df['term'] = df['term'].apply(lambda x: re.sub("[^0-9.,]", '', x))
df['term'] = pd.to_numeric(df['term'])

In [28]:
df['int_rate'] = df['int_rate'].apply(lambda x: re.sub("[^0-9.,]", '', x))
df['int_rate'] = pd.to_numeric(df['int_rate'])

In [29]:
df['emp_length'] = ['0' if x=='< 1 year' else x for x in df['emp_length']]
df['emp_length'] = df['emp_length'].apply(lambda x: re.sub("[^0-9.,]", '', x))
df['emp_length'] = pd.to_numeric(df['emp_length'], errors=coerce)

In [30]:
df['revol_util'] = df['revol_util'].apply(lambda x: re.sub("[^0-9.,]", '', str(x)))
df['revol_util'] = pd.to_numeric(df['revol_util'], errors=coerce)

#### Object to date

In [31]:
obj_date_cols = [
    #'issue_d'
    'earliest_cr_line'
    ]

In [32]:
df[obj_date_cols] = df[obj_date_cols].apply(lambda x: pd.to_datetime(x))

#### Numeric columns

In [33]:
for i in df.select_dtypes(include=[np.number]).columns:
    print ", '{0}'".format(i)

, 'loan_amnt'
, 'term'
, 'int_rate'
, 'installment'
, 'emp_length'
, 'annual_inc'
, 'dti'
, 'delinq_2yrs'
, 'inq_last_6mths'
, 'mths_since_last_delinq'
, 'mths_since_last_record'
, 'open_acc'
, 'pub_rec'
, 'revol_bal'
, 'revol_util'
, 'total_acc'
, 'collections_12_mths_ex_med'
, 'mths_since_last_major_derog'
, 'acc_now_delinq'
, 'total_rev_hi_lim'
, 'acc_open_past_24mths'
, 'avg_cur_bal'
, 'bc_open_to_buy'
, 'bc_util'
, 'chargeoff_within_12_mths'
, 'delinq_amnt'
, 'mo_sin_old_il_acct'
, 'mo_sin_old_rev_tl_op'
, 'mo_sin_rcnt_rev_tl_op'
, 'mo_sin_rcnt_tl'
, 'mort_acc'
, 'mths_since_recent_bc'
, 'mths_since_recent_bc_dlq'
, 'mths_since_recent_inq'
, 'mths_since_recent_revol_delinq'
, 'num_accts_ever_120_pd'
, 'num_actv_bc_tl'
, 'num_actv_rev_tl'
, 'num_bc_sats'
, 'num_bc_tl'
, 'num_il_tl'
, 'num_op_rev_tl'
, 'num_rev_accts'
, 'num_rev_tl_bal_gt_0'
, 'num_sats'
, 'num_tl_120dpd_2m'
, 'num_tl_30dpd'
, 'num_tl_90g_dpd_24m'
, 'num_tl_op_past_12m'
, 'pct_tl_nvr_dlq'
, 'percent_bc_gt_75'
, 'pub

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 134779 entries, 36805548 to 9199665
Data columns (total 64 columns):
loan_amnt                         134779 non-null int64
term                              134779 non-null int64
int_rate                          134779 non-null float64
installment                       134779 non-null float64
grade                             134779 non-null object
sub_grade                         134779 non-null object
emp_length                        128011 non-null float64
home_ownership                    134779 non-null object
annual_inc                        134779 non-null float64
verification_status               134779 non-null object
desc                              12271 non-null object
purpose                           134779 non-null object
dti                               134779 non-null float64
delinq_2yrs                       134779 non-null int64
earliest_cr_line                  134779 non-null datetime64[ns]
inq_last_6mths   

### Dummies for object cols

Turn description into a bin for now

In [40]:
df['desc_bin'] = [1 if x else 0 for x in df['desc']]

In [41]:
df.drop(['desc'], axis=1, inplace=True)

In [42]:
obj_cols = df.select_dtypes(include=[object]).columns

In [43]:
df[obj_cols].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 134779 entries, 36805548 to 9199665
Data columns (total 5 columns):
grade                  134779 non-null object
sub_grade              134779 non-null object
home_ownership         134779 non-null object
verification_status    134779 non-null object
purpose                134779 non-null object
dtypes: object(5)
memory usage: 6.2+ MB


In [44]:
df['grade'].value_counts()

C    38552
B    35317
D    24956
A    18148
E    12355
F     4321
G     1130
Name: grade, dtype: int64

In [45]:
df['sub_grade_number'] = df['sub_grade'].str[1]

In [46]:
df['sub_grade_number'].value_counts()

1    27520
4    27078
3    27012
2    26677
5    26492
Name: sub_grade_number, dtype: int64

In [47]:
df['home_ownership'].value_counts()

MORTGAGE    67615
RENT        54160
OWN         13003
ANY             1
Name: home_ownership, dtype: int64

In [48]:
df['verification_status'].value_counts()

Source Verified    54008
Not Verified       40719
Verified           40052
Name: verification_status, dtype: int64

In [49]:
df['purpose'].value_counts()

debt_consolidation    82994
credit_card           29412
home_improvement       7523
other                  6275
major_purchase         2242
medical                1508
small_business         1461
car                    1073
moving                  890
vacation                788
house                   520
renewable_energy         88
wedding                   5
Name: purpose, dtype: int64

Don't use emp_title (too many uniques), pymnt_plan (all the same), title (too many uniques, seems to have been reconsolidated under purpose, don't use address zip code or state for now

In [51]:
for i in obj_cols:
    df = df.merge(pd.get_dummies(df[i]
                                 , prefix='{0}'.format(i))
                , left_index=True
                , right_index=True)

In [54]:
df.drop(obj_cols, axis=1, inplace=True)

In [58]:
df[df.columns[:df.shape[1]/2]].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 134779 entries, 36805548 to 9199665
Data columns (total 61 columns):
loan_amnt                         134779 non-null int64
term                              134779 non-null int64
int_rate                          134779 non-null float64
installment                       134779 non-null float64
emp_length                        128011 non-null float64
annual_inc                        134779 non-null float64
dti                               134779 non-null float64
delinq_2yrs                       134779 non-null int64
earliest_cr_line                  134779 non-null datetime64[ns]
inq_last_6mths                    134779 non-null int64
mths_since_last_delinq            68833 non-null float64
mths_since_last_record            26035 non-null float64
open_acc                          134779 non-null int64
pub_rec                           134779 non-null int64
revol_bal                         134779 non-null int64
revol_util          

In [59]:
df[df.columns[df.shape[1]/2:]].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 134779 entries, 36805548 to 9199665
Data columns (total 61 columns):
grade_B                                134779 non-null uint8
grade_C                                134779 non-null uint8
grade_D                                134779 non-null uint8
grade_E                                134779 non-null uint8
grade_F                                134779 non-null uint8
grade_G                                134779 non-null uint8
sub_grade_A1                           134779 non-null uint8
sub_grade_A2                           134779 non-null uint8
sub_grade_A3                           134779 non-null uint8
sub_grade_A4                           134779 non-null uint8
sub_grade_A5                           134779 non-null uint8
sub_grade_B1                           134779 non-null uint8
sub_grade_B2                           134779 non-null uint8
sub_grade_B3                           134779 non-null uint8
sub_grade_B4               